In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=bca140e8518222855d13c7e8a2a41a0a98142a499e3562e3cfb1fc69105f3f90
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Import libreries

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import ChiSqSelector
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator

Upload database on the CSV file or other compatible dataset file to Colab.Create a SparkSession

In [ ]:
spark = SparkSession.builder.appName("SalaryPrediction").getOrCreate()

Read database

In [ ]:
data = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("Salary_dataset.csv")

In [ ]:
data.show(3)

+---+------------------+-------+
|_c0|   YearsExperience| Salary|
+---+------------------+-------+
|  0|1.2000000000000002|39344.0|
|  1|1.4000000000000001|46206.0|
|  2|               1.6|37732.0|
+---+------------------+-------+
only showing top 3 rows



Calculed mean years experience

In [ ]:
meanYearsExperience = data.select("YearsExperience").agg({"YearsExperience": "mean"}).collect()[0][0]
print(f"Mean YearsExperience: {meanYearsExperience}")

Mean YearsExperience: 5.413333333333332


Vectorize Features - Combine the selected features into a single vector using

In [ ]:
feature = ['YearsExperience']
label = 'Salary'
Vector  = VectorAssembler(inputCols=feature , outputCol="feature")

randomSplit([0.7 , 0.3]) Training set: 80 data points (80% of the original data) Testing set: 20 data points (20% of the original data)

In [ ]:
df = Vector.transform(data)
(x_train , x_test)  = df.randomSplit([0.7 ,0.3])

Model LinearRegression

In [ ]:
linear = LinearRegression(featuresCol="feature", labelCol=label)
# Train the model on the training data
model = linear.fit(x_train)

Make predictions on the testing data

In [ ]:
predictions = model.transform(x_test)

In [ ]:
evaluator = RegressionEvaluator(labelCol=label, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

The results

In [ ]:
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 5421.582162092105


The model coefficients and intercept

In [ ]:
print(f"Coefficients:{model.coefficients}")
print(f"Intercept: {model.intercept}")

Coefficients:[9610.355807116148]
Intercept: 23556.231264625265


In [ ]:
workExperience = 5  # Years of experience
featuresVector = Vector.transform(spark.createDataFrame([(workExperience, )], ["YearsExperience"])).select("feature").collect()[0].feature
predictedSalary = model.transform(spark.createDataFrame([(featuresVector, )], ["feature"])).select("prediction").collect()[0].prediction
print(f"Predicted salary for {workExperience} years of experience: {predictedSalary}")
# Stop the SparkSession
spark.stop()

Predicted salary for 5 years of experience: 71608.010300206
